In [1]:
import torch
import torch.nn as nn
from torch.utils.data import Dataset, DataLoader
import torchvision
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
import torch.optim as optim
import torch.nn.functional as F
from tqdm.notebook import tqdm


In [2]:
class CardDataset(Dataset):
    def __init__(self, dir_path, transform=None):
        self.data = ImageFolder(dir_path, transform = transform)

    def __len__(self):
        return len(self.data)

    def __getitem__(self, idx):
        return self.data[idx]
    

In [3]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor()
])

In [4]:
train_path = '/aul/homes/amaha038/DeepLearning/Datasets/Card_Dataset_Kaggle/train'
val_path = '/aul/homes/amaha038/DeepLearning/Datasets/Card_Dataset_Kaggle/valid/'

train_data = CardDataset(train_path, transform = transform)
val_data = CardDataset(val_path, transform = transform)

In [5]:
train_loader = DataLoader(train_data, batch_size = 32, shuffle=True)
val_loader = DataLoader(val_data, batch_size = 32, shuffle=False)

In [ ]:
#Formula for Convolution Layer Operation
"""
nn.conv2d = (W - F + 2P)/S + 1,
where W = width of the input (image)
F = Filter size (kernel)
P = Padding
S = stride 
"""
#In our case
"""
Width = 224
F = 3
S = 1
P = 0

Hence, the output of self.conv1 = (224-3+0)/1 + 1 = 222
"""
#Note By Default: torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride=1, padding=0)

In [6]:
num_classes = 53
class SimpleCNNClassifier(nn.Module):
    def __init__(self, num_classes):
        super(SimpleCNNClassifier, self).__init__()

        self.conv1 = nn.Conv2d(3, 6, 3) # in_channels=3, out_channels=16, kernel_size=3 It gives 222
        self.pool = nn.MaxPool2d(2, 2) # image size 6*111*111(222/2)
        self.conv2 = nn.Conv2d(6, 16,3)  #  (111 - 3 + 1) = 109
        # pooling again with 16*54*54 (109/2)
        self.fc1 = nn.Linear(16*54*54, 256) #256 is selected manually
        self.fc2 = nn.Linear(256, num_classes)

    def forward(self, x):
        x = self.pool(F.relu(self.conv1(x))) # (32, 6, 111, 111) #(Batchsize, channel, dimension_height, dimension_width)
        x = self.pool(F.relu(self.conv2(x))) # (32, 16, 54, 54)
        x = x.view(x.size(0), -1) # Flatten: (32, 16*54*54)
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        return x 
        

In [7]:
model = SimpleCNNClassifier(num_classes)
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr = 0.001)

In [9]:
#training
epochs = 2
train_losses, val_losses = [], []
best_val_acc = 0.0
best_val_loss = float('inf')

for epoch in range(epochs):

    #training
    model.train()
    running_loss = 0.0
    correct_train = 0
    total_train = 0

    for images, labels in tqdm(train_loader, desc='Training loop'):
        images, labels = images.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(images)
        loss = criterion(outputs, labels)

        #prediction
        _, preds = torch.max(outputs, 1)
        correct_train += (preds == labels).sum().item()
        total_train += labels.size(0)

        loss.backward()
        optimizer.step()
        running_loss += loss.item() * labels.size(0) # loss.item() gives the average loss per image in the current batch
    train_loss = running_loss / len(train_loader.dataset)
    train_losses.append(train_loss)
    train_acc = correct_train / total_train

    #validation phase

    model.eval()
    running_loss = 0.0
    correct_val = 0
    total_val = 0

    with torch.no_grad():
        for images, labels in tqdm(val_loader, desc='Validation loop'):
            images, labels = images.to(device), labels.to(device)

            outputs = model(images)
            loss = criterion(outputs, labels)

            _, preds = torch.max(outputs, 1)
            correct_val += (preds == labels).sum().item()
            total_val += labels.size(0)
            running_loss += loss.item() * labels.size(0)
    val_loss = running_loss / len(val_loader.dataset)
    val_losses.append(val_loss)
    val_acc = correct_val / total_val

    print(f"Epoch {epoch+1}/{epochs} - Train loss: {train_loss}, Val Loss: {val_loss}. Train Acc.: {train_acc}, Val Acc.: {val_acc}")

    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), './weights/best_val_acc.pth')
        print(f"The model weight is saved based on val_acc: {best_val_acc:.4f}")

    if val_loss < best_val_loss:
        best_val_loss = val_loss
        torch.save(model.state_dict(), './weights/best_val_loss.pth')
        print(f"The model weight is saved based on val_loss: {best_val_loss:.4f}")
        
    

Training loop:   0%|          | 0/239 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/9 [00:00<?, ?it/s]

Epoch 1/2 - Train loss: 2.934760473957842, Val Loss: 1.609550942115064. Train Acc.: 0.2433105981112277, Val Acc.: 0.5245283018867924
The model weight is saved based on val_acc: 0.5245
The model weight is saved based on val_loss: 1.6096


Training loop:   0%|          | 0/239 [00:00<?, ?it/s]

Validation loop:   0%|          | 0/9 [00:00<?, ?it/s]

Epoch 2/2 - Train loss: 1.6072091625970157, Val Loss: 1.2928998326355556. Train Acc.: 0.5661070304302204, Val Acc.: 0.630188679245283
The model weight is saved based on val_acc: 0.6302
The model weight is saved based on val_loss: 1.2929


In [10]:
test_path = "/aul/homes/amaha038/DeepLearning/Datasets/Card_Dataset_Kaggle/test"

test_data = CardDataset(test_path, transform=transform)

test_loader = DataLoader(test_data, batch_size=32, shuffle=False)


In [11]:
model = SimpleCNNClassifier(num_classes=53)
model.load_state_dict(torch.load('./weights/best_val_acc.pth'))
model.to(device)
                    

/tmp/ipykernel_1331899/415696116.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.load_state_dict(torch.load('./weights/best_val_acc.pth'))


SimpleCNNClassifier(
  (conv1): Conv2d(3, 6, kernel_size=(3, 3), stride=(1, 1))
  (pool): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
  (conv2): Conv2d(6, 16, kernel_size=(3, 3), stride=(1, 1))
  (fc1): Linear(in_features=46656, out_features=256, bias=True)
  (fc2): Linear(in_features=256, out_features=53, bias=True)
)

In [20]:
model.eval()

total_loss = 0.0
total_acc = 0
total_test = 0
with torch.no_grad():
    for images, labels in test_loader:
        images, labels = images.to(device), labels.to(device)
        outputs = model(images)
        loss = criterion(outputs, labels)

        _, preds = torch.max(outputs, 1)
        total_acc += (preds==labels).sum().item()
        total_test += labels.size(0)

        total_loss += loss.item() * labels.size(0)
test_acc = total_acc / total_test
test_loss = total_loss / total_test

print(f"Total test loss: {test_loss:.4f} and total test accuracy: {test_acc:.4f}")


Total test loss: 1.3175 and total test accuracy: 0.6302
